In [1]:
%env CUDA_VISIBLE_DEVICES = 1

env: CUDA_VISIBLE_DEVICES=1


In [2]:
%load_ext autoreload
%autoreload 2
import sys; 
sys.path.extend(['/home/meet/FlowMatchingTests/conditional-flow-matching/'])

import matplotlib.pyplot as plt
import numpy as np
import torch
from torchcfm.conditional_flow_matching import *
from physics_flow_matching.unet import UNetModel
from torchdiffeq import odeint
from einops import rearrange

In [3]:
label = np.load("/home/xiantao/case/wall_pressure/bigchannel/data/to_meet/channel_180_w_y5-150_41-220_out.npy")
test_label = np.load("/home/xiantao/case/wall_pressure/bigchannel/data/to_meet/channel_180_w_y5-150_221-259_out.npy")
m, std = label.mean(axis=(0,2,3), keepdims=True), label.std(axis=(0,2,3), keepdims=True)
# test_label = rearrange(test_label, "b l c w -> (b c) l w")
# test_label = (test_label[:, 160:])[:, None]
# Y = (label - label.mean())/label.std()

In [4]:
a, b, c = np.load("/home/xiantao/case/wall_pressure/bigchannel/data/to_meet/channel_180_wall_dudy_41-220_out.npy"), np.load("/home/xiantao/case/wall_pressure/bigchannel/data/to_meet/channel_180_wall_dwdy_41-220_out.npy"), np.load("/home/xiantao/case/wall_pressure/bigchannel/data/to_meet/channel_180_wall_pressure_41-220_out.npy")
data = np.concatenate((c,a[:,None],b[:,None]), axis=1)
m_, std_ = data.mean(axis=(0,2,3), keepdims=True), data.std(axis=(0,2,3), keepdims=True)

In [5]:
a, b, c = np.load("/home/xiantao/case/wall_pressure/bigchannel/data/to_meet/channel_180_wall_dudy_221-259_out.npy"), np.load("/home/xiantao/case/wall_pressure/bigchannel/data/to_meet/channel_180_wall_dwdy_221-259_out.npy"), np.load("/home/xiantao/case/wall_pressure/bigchannel/data/to_meet/channel_180_wall_pressure_221-259_out.npy")
a, b= a[:, None], b[:, None]
test_data = np.concatenate((c,a,b), axis=1)
X = (test_data - m_)/std_

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
exp=9
iteration = 5
ot_cfm_model = UNetModel(dim=[3, 160, 200],
                         channel_mult=None,
                         out_channels=1,
                         num_channels=128,
                         num_res_blocks=2,
                         num_head_channels=64,
                         attention_resolutions="40, 20",
                         dropout=0.2,
                         use_new_attention_order=True,
                        use_scale_shift_norm=True)
state = torch.load(f"/home/meet/FlowMatchingTests/conditional-flow-matching/physics_flow_matching/wm_vf/exps/exp_{exp}/saved_state/checkpoint_{iteration}.pth")
ot_cfm_model.load_state_dict(state["model_state_dict"])
ot_cfm_model.to(device)
ot_cfm_model.eval();

/tmp/ipykernel_3502779/3326358270.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(f"/home/meet/FlowMatchingTests/conditional-flow-matching/physics_fl

In [8]:
batch_size=10

In [14]:
wall_norm_dict = {0:5, 1:150}
key = 1

In [15]:
samples_list = []
for batch in range(X.shape[0]//batch_size):
    with torch.no_grad():
        out = ot_cfm_model(torch.tensor(wall_norm_dict[key]*np.ones(batch_size).astype(np.float32), device=device)[:, None], torch.tensor(X[batch*batch_size:(batch+1)*batch_size], device=device)).detach().cpu().numpy()
    out *= std[:, key]
    out += m[:, key]
    samples_list.append(out)

In [16]:
samples = np.concatenate(samples_list)

In [18]:
np.save(f"/home/meet/FlowMatchingTests/conditional-flow-matching/physics_flow_matching/wm_vf/exps/exp_{exp}/y_{wall_norm_dict[key]}_samples_{iteration}epch", samples[:, 0])

In [17]:
# for i in range(5):
#     fig,axes = plt.subplots(nrows=1, ncols=2)
#     ax = axes[0].imshow(samples[i, 0])
#     plt.colorbar(ax,ax=axes[0])
#     ax = axes[1].imshow(test_label[3000+i, 0])
#     ax = plt.colorbar(ax,ax=axes[1])

In [14]:
# for i in range(0,100,5):
#     plt.figure()
#     plt.imshow(out[i, 0, 0])

In [15]:
# np.save(f"/home/meet/FlowMatchingTests/conditional-flow-matching/physics_flow_matching/wp_ws/exps/FM_exps/exp_{exp}/samples_{iteration}iter_7000", samples[:, 0])